In [1]:
import numpy as np  
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt  
import math  

Using TensorFlow backend.


In [2]:
#Dimensoes de entrada da VGG16
img_width, img_height = 224, 224  

batch_size = 16

#Path dos dados a serem utilizados
data_dir40 = '../Dados/breast_cancer40'
data_dir100 = '../Dados/breast_cancer100'
data_dir200 = '../Dados/breast_cancer200'
data_dir400 = '../Dados/breast_cancer400'

# Carregar as features de magnitude 40x 

In [48]:
datagen = ImageDataGenerator(rescale=1. / 255)
#Carregando fator 40x
generator40 = datagen.flow_from_directory(
    data_dir40,
    batch_size=batch_size,
    target_size=(img_width, img_height),
    class_mode='categorical',
    shuffle=False)

#Features extraidas
data40 = np.load('bottleneck_features_40.npy')
len_Data_40 = len(data40)
data40 = data40.reshape(len_Data_40,-1)
#Labels em ordem dos dados
labels40 = generator40.classes
num_classes40 = len(generator40.class_indices)

Found 1995 images belonging to 2 classes.


# Dividir o conjunto de dados em treino e teste com 80% dos dados para treino

In [20]:
from sklearn.model_selection import train_test_split

X40_train, X40_test, y40_train, y40_test = train_test_split(data40, labels40, random_state=0, train_size=0.8)

len_train_40 = len(X40_train)
X40_train = X40_train.reshape(len_train_40,-1)

len_test_40 = len(X40_test)
X40_test = X40_test.reshape(len_test_40,-1)

## Teste dos dados com múltiplos classificadores

In [7]:
from sklearn import svm
clfsvm = svm.SVC()

In [21]:
clfsvm.fit(X40_train,y40_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
clfsvm.score(X40_test,y40_test)

0.66416040100250628

In [23]:
clf = svm.SVC()

In [24]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X40_train, y40_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.69 (+/- 0.00)


In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X40_train, y40_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.69 (+/- 0.00)


In [26]:
linear_svm = svm.SVC(kernel='linear')

In [27]:
scores = cross_val_score(linear_svm, X40_train, y40_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.85 (+/- 0.02)


In [28]:
scores = cross_val_score(linear_svm, X40_train, y40_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.86 (+/- 0.04)


In [29]:
linear_svm.fit(X40_train,y40_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [30]:
linear_svm.score(X40_test,y40_test)

0.84962406015037595

In [35]:
polynomial_svm = svm.SVC(kernel='poly')
sigmoid_svm = svm.SVC(kernel='sigmoid')

In [36]:
scores = cross_val_score(polynomial_svm, X40_train, y40_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.69 (+/- 0.00)


In [37]:
scores = cross_val_score(sigmoid_svm, X40_train, y40_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.69 (+/- 0.00)


In [38]:
polynomial_svmmial_svm.fit(X40_train,y40_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
polynomial_svm.score(X40_test,y40_test)

0.66416040100250628

In [40]:
sigmoid_svm.fit(X40_train,y40_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
sigmoid_svm.score(X40_test,y40_test)

0.66416040100250628

## Utilizando GradientBoosting

In [43]:
from sklearn.ensemble import GradientBoostingClassifier

In [44]:
gb_deviance = GradientBoostingClassifier()

In [45]:
scores = cross_val_score(gb_deviance, X40_train, y40_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84 (+/- 0.03)


In [46]:
gb_exp = GradientBoostingClassifier(loss='exponential')

In [47]:
scores = cross_val_score(gb_exp, X40_train, y40_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.83 (+/- 0.03)


In [49]:
scores = cross_val_score(linear_svm, data40, labels40, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.87 (+/- 0.02)
